$\DeclareMathOperator{\opdiv}{div}$
$\DeclareMathOperator{\opcurl}{curl}$

Reissner Mindlin Plates
====

We search for the vertical deflection $w$ and the linearized rotation vector $\beta$. The internal energy is

$$
\| \varepsilon(\beta) \|_{L_2}^2 + \frac{1}{t^2} \| \nabla w - \beta \|_{L_2}
$$

where $t$ is the thickness of the plate. For $t \rightarrow 0$, this can be understood as a penalty formulation for the shear-free constraint

$$
\beta = \nabla w
$$

In the limit case, the internal energy is 

$$
\| \varepsilon(\nabla w) \|_{L_2}^2,
$$

and we have the Kirchhoff plate model.

We are interested in positive, but small parameters $t$. If there are not enough finite element functions such that the discrete Kirchhoff constraint

$$
\beta_h = \nabla w_h
$$

can be satisfied, we observe bad results known as shear locking.

Now, we apply the TDNNS method to discretize the bending term $\| \varepsilon(\beta) \|_{L_2}^2$. We discretize the rotation in the Nedelec space. Thus, all discrete gradients $\nabla w_h$ are in the space of rotations, and we don't have the locking problem.

Find: $\sigma \in H(\opdiv \opdiv)$, $\beta \in H(\opcurl)$, $w \in H^1$ such that

$$
\begin{array}{ccccll}
\int A \sigma : \tau & + & \left< \opdiv \tau, \beta \right> & = & 0  \quad & \forall \, \tau \\
\left< \opdiv \sigma, \delta \right> & - & t^2 \int (\nabla w - \beta) (\nabla v - \delta) & = & \int f v \quad & \forall \, v, \delta 
\end{array}
$$

In [1]:
from ngsolve import *
from netgen.geom2d import unit_square
import netgen.gui

In [2]:
mesh = Mesh (unit_square.GenerateMesh(maxh=0.1))
Draw (mesh)

order = 2
Sigma = HDivDiv(mesh, order=order, dirichlet=".*")
W = H1(mesh, order=order+1, dirichlet=".*")
V = HCurl(mesh, order=order, dirichlet=".*")
X = FESpace([Sigma,W,V])

In [3]:
sigma, w, beta = X.TrialFunction()
tau, v, delta = X.TestFunction()

thick = 0.01

n = specialcf.normal(2)
def tang(u): return u-(u*n)*n

a = BilinearForm(X, symmetric=True)
a += SymbolicBFI (InnerProduct (sigma, tau) + div(sigma)*delta 
                  + div(tau)*beta)
a += SymbolicBFI (-(sigma*n) * tang(delta) - (tau*n)*tang(beta), 
                  element_boundary=True)
a += SymbolicBFI (-thick**2*(grad(w)-beta)*((grad(v)-delta)))
a.Assemble()

f = LinearForm(X)
f += SymbolicLFI (500 * v)
f.Assemble()

u = GridFunction(X)
u.vec.data = a.mat.Inverse(X.FreeDofs()) * f.vec

Draw (u.components[0], mesh, name="sigma")
Draw (u.components[1], mesh, name="disp")
Draw (u.components[2], mesh, name="rotation")